In [1]:
#STEP ONE : IMPORTS

In [13]:
import pandas as pd
import nltk
from nltk.tokenize import word_tokenize
from collections import Counter

from sklearn.model_selection import train_test_split 

In [3]:
#STEP 2 : DATA COLLECTION AND CLEANING

In [4]:
english_french = pd.read_csv('C:/Users/user/Desktop/AI and Data Science Workshop/MyNLPModel/data/eng_-french.csv')

print(english_french.head())

  English words/sentences French words/sentences
0                     Hi.                 Salut!
1                    Run!                Cours !
2                    Run!               Courez !
3                    Who?                  Qui ?
4                    Wow!             Ça alors !


In [5]:
english_french['French words/sentences'] = english_french['French words/sentences'].str.lower()
english_french['English words/sentences'] = english_french['English words/sentences'].str.lower()

In [6]:
english_french.head()

,English words/sentences,French words/sentences
0,hi.,salut!
1,run!,cours !
2,run!,courez !
3,who?,qui ?
4,wow!,ça alors !


In [7]:
#STEP 3 : TOKENIZING THE DATA

In [8]:
english_french['English_tokenized'] = english_french['English words/sentences'].apply(word_tokenize)
english_french['French_tokenized'] = english_french['French words/sentences'].apply(word_tokenize)


In [9]:
#STEP 4 : BUILDING THE VOCABULARY

In [10]:
# Build vocabulary for English
English_vocab = Counter(word for sentence in english_french['English_tokenized'] for word in sentence)
English_word_to_index = {word: idx for idx, (word, _) in enumerate(English_vocab.items(), 1)}
English_index_to_word = {idx: word for word, idx in English_word_to_index.items()}

# Build vocabulary for French
French_vocab = Counter(word for sentence in english_french['French_tokenized'] for word in sentence)
French_word_to_index = {word: idx for idx, (word, _) in enumerate(French_vocab.items(), 1)}
French_index_to_word = {idx: word for word, idx in French_word_to_index.items()}

In [11]:
#STEP 5 : ENCODING; CONVERTING THE SENTENCES INTO SEQUENCES OF INTEGERS BASED ON THE VOCABULARY.

In [12]:
# Encode English sentences
english_french['English_encoded'] = english_french['English_tokenized'].apply(lambda x: [English_word_to_index[word] for word in x])

# Encode French sentences
english_french['French_encoded'] = english_french['French_tokenized'].apply(lambda x: [French_word_to_index[word] for word in x])

In [ ]:
#STEP 6 : TRAIN-TEST-SPLITTING THE DATA FOR TESTING.

In [ ]:
x = 
y = 

In [ ]:
#STEP 7 : BUILDING THE MODEL

In [ ]:
#STEP 8 : TESTING MACHINE PREDICTIONS.

In [ ]:
#TRANSLATING USER INPUT FROM ENGLISH TO FRENCH.

# Predict and Translate User Input
def preprocess_input(sentence):
    tokens = word_tokenize(sentence.lower())
    token_ids = [English_word_to_index.get(word, 0) for word in tokens]  # Use 0 for unknown words
    padded_token_ids = pad_sequences([token_ids], maxlen=max_len_english, padding='post')
    return padded_token_ids

def decode_sequence(input_seq):
    decoded_sentence = []
    for idx in input_seq:
        word = French_index_to_word.get(idx, '')
        if word:
            decoded_sentence.append(word)
    return ' '.join(decoded_sentence)

def translate_to_french(sentence):
    preprocessed_input = preprocess_input(sentence)
    prediction = model.predict(preprocessed_input)
    predicted_token_ids = prediction.argmax(axis=-1)[0]  # Get the token IDs with the highest probability
    translated_sentence = decode_sequence(predicted_token_ids)
    return translated_sentence

# Example usage
user_input = input("Enter an English sentence to translate: ")
Translated_sentence = Translate_to_french(user_input)
print(f"Translated to French: {Translated_sentence}")